# JPX Competition Metric Definitionの日本語訳
* **このnotebookは運営による評価指標の説明である[JPX Competition Metric Definition](https://www.kaggle.com/code/smeitoma/jpx-competition-metric-definition) を和訳したものです。**
* 自分のために雑に訳したものですので、多少の翻訳ミスはご容赦ください。
* 参考までに[日本語版のWikipediaによるシャープレシオの説明](https://ja.wikipedia.org/wiki/%E3%82%B7%E3%83%A3%E3%83%BC%E3%83%97%E3%83%BB%E3%83%AC%E3%82%B7%E3%82%AA)です。
* **このnotebookが役に立ったら、[元のnotebook](https://www.kaggle.com/code/smeitoma/jpx-competition-metric-definition)へのupvoteをお願いします。**

# JPX Competition の評価指標

In this competition, the following conditions set will be used to compete for scores.
> このコンペティションでは、次の条件でスコアを競います。

1. The model will use the closing price ($C_{(k, t)}$) until that business day ($t$) and other data every business day as input data for a stock ($k$), and predict rate of change ($r_{(k, t)}$) of closing price of the top 200 stocks and bottom 200 stocks on the following business day ($C_{(k, t+1)}$) to next following business day ($C_{(k, t+2)}$)
> このモデルは、ある銘柄($k$)について、その営業日($t$)までの終値($C_{(k, t)}$) と毎営業日のその他のデータを入力データとして、翌営業日の上位200銘柄と下位200銘柄の終値 ($C_{(k, t+1)}$) と翌々営業日 ($C_{(k, t+2)}$ の変化率 ($r_{(k, t)}$) を予想するものです。

    $$
    r_{(k, t)} = \frac{C_{(k, t+2)} - C_{(k, t+1)}}{C_{(k, t+1)}}
    $$
    
2. Within top 200 stock predicted ($up_i\;\;(i = 1, 2, \ldots, 200)$), multiply by their respective rate of change with linear weights of 2-1 for rank 1-200 and denote their sum as $S_{up}$.
>上位200銘柄の予測値($up_i; \;(i = 1, 2, \ldots, 200)$)に、1-200位までの順位に応じた線形重み2~1でそれぞれの変化率を乗じ、その和を$S_{up}$と表記します。

    $$
    S_{up} = \frac{\sum^{200}_{i=1}(r_{({up_i}, t)} * linear function(2, 1)_i))}{Average(linear function(2, 1))}
    $$
    
3. Within bottom 200 stocks predicted  ($down_i\;\;(i = 1, 2, \ldots, 200)$), multiply by their respective rate of change with linear weights of 2-1 for bottom rank 1-200 and denote their sum as $S_{down}$.
> 下位200銘柄の予測値($down_i; \;(i = 1, 2, \ldots, 200)$)に、下位1～200位までの順位に応じた線形重み2~1でそれぞれの変化率を乗じ、その和を$S_{down}$と表記します。

    $$
    S_{down} = \frac{\sum^{200}_{i=1}(r_{({down_i}, t)} * linear function(2, 1)_i)}{Average(linear function(2, 1))}
    $$
    
4. The result of subtracting $S_{down}$ from $S_{up}$ is $R_{day}$ and is called "**daily spread return**".
> $S_{up}$から$S_{down}$を引いた結果が$R_{day}$で、これを「**日次スプレッドリターン**」と呼びます。

    $$
    R_{day} = S_{up} - S_{down}
    $$
    
5. The daily spread return is calculated every business day during the public/private period and obtained as a time series for that period. The mean/standard deviation of the time series of daily spread returns is used as the score. Score calculation formula (x is the business day of public/private period)
> 日次スプレッドリターンは、公開/非公開期間中の毎営業日に算出し、その期間の時系列として取得します。日次スプレッドリターンの時系列の平均／標準偏差をスコアとして使用します。スコア算出式は以下の通りです。（xは公開/非公開期間の営業日）

    $$
    Score = \frac{Average(R_{day_1-day_x})}{STD(R_{day_1-day_x})}
    $$
    
6. The Kagger with the largest score for the private period wins.
> privateな期間においてスコアが最も大きいKagglerが優勝です。

# Evaluation function code

In [ ]:
import numpy as np
import pandas as pd


def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results(予想値)
        portfolio_size (int): # of equities to buy/sell(購入/売却する株式の数)
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.(最も高い順位の銘柄と最も低い順位の銘柄の相対的な重みを表します。)
    Returns:
        (float): sharpe ratio(シャープレシオ)
    """
    def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
        """
        Args:
            df (pd.DataFrame): predicted results(予想値)
            portfolio_size (int): # of equities to buy/sell(購入/売却する株式の数)
            toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.(最も高い順位の銘柄と最も低い順位の銘柄の相対的な重みを表します。)
        Returns:
            (float): spread return(スプレッドリターン)
        """
        assert df['Rank'].min() == 0
        assert df['Rank'].max() == len(df['Rank']) - 1
        weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
        purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
        short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
        return purchase - short

    buf = df.groupby('Date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio

The following rules are used to determine which stocks are available for investment.
> 投資対象銘柄は、以下のルールで決定されます。

* The top 2,000 common stocks by market capitalization that have been listed for at least one year as of 2021-12-31 are eligible for investment.
> 2021-12-31 時点で上場後 1 年以上経過した時価総額上位 2,000 銘柄が投資対象となります。

* If a stock is designated as Securities Under Supervision or Securities to Be Delisted during the private period, it will be excluded from investment after the date of designation.
> 非公開期間中に監理銘柄、上場廃止銘柄に指定された場合、指定日以降は投資対象から除外されます。

* When calculating the score, the adjusted stock price is used.
> スコア算出の際は、修正株価を使用します。

# Intentions of problem
> 問題の意図

In general, it is not possible to assume that data will have the same distribution permanently in two different periods of financial market time-series data. For example, the nature of the market changed dramatically between February 2020 and March 2020 and beyond due to changes in global conditions caused by COVID-19 and other factors.
> 一般に、金融市場の時系列データにおいて、2つの異なる時期のデータが永続的に同じ分布になることは想定できません。例えば、2020年2月と2020年3月以降では、COVID-19などによる世界情勢の変化により、市場の性質が大きく変化しています。

In the case of a competition that focuses on a financial market with shifting data distribution characteristics, we thought that the winner of the competition should be the Kaggler who constructed a robust model that does not depend on changes in the data distribution.
> このようにデータ分布の特性が変化する金融市場を対象としたコンペティションの場合、データ分布の変化に依存しないロバストなモデルを構築したKagglerが勝者となるべきであると考えたのです。

Based on the above assumptions, the following were considered in the design of this competition
> 上記の前提のもと、このコンペティションの設計にあたっては、以下を考慮しました。

* The number of stocks to be forecast each business day is the difference between the rate of change of 200 stocks, each of which is 10% or more than the number of stocks to be invested in (2000 stocks), so that the performance of the model can be competed without being affected by the events of individual stocks. In practice, however, institutional investors and funds often invest in 50-100 stocks, so there is a slight deviation from the real-world setting of the problem.
> 毎営業日の予測対象銘柄数は、投資対象銘柄数（2000銘柄）に対して、各銘柄の変動率が10％以上となる200銘柄の差分とし、個別銘柄のイベントに左右されずにモデルのパフォーマンスを競うことができるようにしました。しかし、実際には機関投資家やファンドが50～100銘柄に投資することが多く、現実の問題設定とは若干のずれがあります。

* When calculating the daily spread return, a linear weight of 2 to 1 is applied to the 1-200 stocks, so that stocks with higher rates of return are placed in the first position.
> 日次スプレッドリターンを計算する際、1-200銘柄に対して2~1の線形加重をかけ、収益率の高い銘柄が1位になるようにしています。

* Since "risk control" is also an important element of investment, the competing score is the **mean/standard deviation** of the time series of daily spread returns, rather than the simple mean or sum of daily spread returns. This makes it necessary to build a model that can respond to changes in the distribution of data and produce stable and high performance, rather than a model that only wins big on certain days.
> 「リスクコントロール」も投資の重要な要素であるため、競合スコアは日次スプレッドリターンの単純な平均や合計ではなく、日次スプレッドリターンの時系列の**平均/標準偏差**とします。そのため、特定の日にだけ大きく勝つモデルではなく、データの分布の変化に対応し、安定した高いパフォーマンスを発揮できるモデルを構築する必要があります。

* The competition also provides option data and other data that can provide clues for estimating the volatility and risk factors of the market itself. These data may be used for more sophisticated risk control. Since the bottom 200 stocks are also included in the forecast, it is possible to adopt a market-neutral strategy (it is also possible to intentionally bias the beta toward the long side).
> このコンペティションでは、市場自体のボラティリティやリスク要因を推定する手がかりとなるオプションデータ等も提供されます。これらのデータは、より高度なリスクコントロールに利用することができます。下位200銘柄も予想に含まれるため、マーケットニュートラルな戦略を採ることも可能です（意図的にベータをロングサイドに偏らせることも可能）。